In [0]:
!pip install xlrd==1.2.0
!pip install openpyxl
!pip install geopandas
!pip install geopy
!pip install geocoder
!pip install reverse_geocoder

Requirement already satisfied: xlrd==1.2.0 in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: openpyxl in /databricks/python3/lib/python3.8/site-packages (3.0.9)
Requirement already satisfied: et-xmlfile in /databricks/python3/lib/python3.8/site-packages (from openpyxl) (1.1.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geopandas in /databricks/python3/lib/python3.8/site-packages (0.10.2)
Requirement already satisfied: fiona>=1.8 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.21)
Requirement already satisfied: shapely>=1.6 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.0)
Requirement already satisfied: pyproj>=2.2.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (3.3.0)
Requirement already satisfied: pandas>=0.25.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.2.4)
Requirement already satisfied: six>=1.7 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.15.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from fiona>=1.8->geopandas) (52.0.0)
Requirement already satisfied: click>=4.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (8.0.3)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2020.12.5)
Requirement already satisfied: munch in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2.5.0)
Requirement already satisfied: cligj>=0.5 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (0.7.2)
Requirement already satisfied: click-plugins>=1.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.1.1)
Requirement already satisfied: attrs>=17 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (21.2.0)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (2.8.1)
 WARNING: You are using pip version 21.0.1; however, version 22.0.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geopy in /databricks/python3/lib/python3.8/site-packages (2.2.0)
Requirement already satisfied: geographiclib<2,>=1.49 in /databricks/python3/lib/python3.8/site-packages (from geopy) (1.52)
 WARNING: You are using pip version 21.0.1; however, version 22.0.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geocoder in /databricks/python3/lib/python3.8/site-packages (1.38.1)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from geocoder) (1.15.0)
Requirement already satisfied: ratelim in /databricks/python3/lib/python3.8/site-packages (from geocoder) (0.1.6)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from geocoder) (8.0.3)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from geocoder) (2.25.1)
Requirement already satisfied: future in /databricks/python3/lib/py

In [0]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta
import os
import warnings
import xlrd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import requests
import json
import geopandas
from geopy import geocoders
from geopy.geocoders import Photon
import geocoder
from numpy import cos, sin, arcsin, sqrt
from math import radians
import scipy.stats as stats
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import reverse_geocoder as rg
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
data = spark.read.table('monthly_extreme_weather_events_data_07_02_v1')
location_data = data.toPandas()

In [0]:
gn = geocoders.GeoNames(username='enter user name')

In [0]:
bing_api_key = 'AvvbqIxh4-maWrS0y2umBoeRVYxQdk4RoWbc5TZaDJXaDuJFqZiyou5Gd_dAboWj'

In [0]:
def get_lat_long(location):
  g = geocoder.bing(location, key=bing_api_key)
  results = g.json
  return results['lat'], results['lng']

In [0]:
location_data_v1 = location_data[['updated_VS_ID', 'updated_VS_location', 'updated_VS_region']].drop_duplicates()

In [0]:
location_data_v1.shape

Out[8]: (148, 3)

In [0]:
for index, row in location_data_v1.iterrows():
  print(row['updated_VS_ID'])
  location = row['updated_VS_location']+', '+row['updated_VS_region']
  latitude, longitude = get_lat_long(location)
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'updated_VS_lat'] = latitude
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'updated_VS_long'] = longitude

100874124
101067666
100678777
100164975
100829721
100780593
100364181
100679984
100532526
100350126
100433484
101257366
100939812
100889724
100920769
100821741
100366057
100907828
100540727
100368030
100365002
100365080
100368383
101214198
100361195
100714421
100714422
100922747
100350133
100064893
101201540
100679910
100679844
100922777
101071955
100845535
100679820
100922933
100680091
100679315
100920044
100855588
100910308
100893951
100679002
100074079
100829704
100855106
100957971
101046438
100878274
100740775
100907759
100679818
100939740
100856432
100369328
100974854
100920029
100366383
100172014
100827735
100183514
100679810
101233342
100726054
100364937
100367662
100669328
100920025
100829759
100907817
100874486
100364657
100678689
100669778
100735436
100410746
100679817
100679003
101064810
100680088
100920891
100364823
100455922
100568204
100366190
100364965
100853829
100673452
100820858
100613872
100359218
100885237
100358246
100364779
100363338
100939861
100875265
100380951
100865704
100351217
100356603
100939830
100907815
100740777
100894811
100922657
100357317
100791840
100960101
100829720
100454797
100663389
100939813
100874165
100657704
100907802
100364230
100589650
100885473
100355048
100669928
100365055
100898729
100099158
100455353
100922815
100783868
100531472
100948925
100737783
100922871
100874123
101140876
100935737
100854873
100351791
100678954
100922612
100351456
100836315
100680016
100920560
100366340
100937736
100922666
100873732

In [0]:
def reverseGeocode(coordinates):
    result = rg.search(coordinates)
    return result

In [0]:
location_data_v2 = location_data[['updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 
                                  'updated_VS_lat', 'updated_VS_long']].drop_duplicates()

In [0]:
for index, row in location_data_v2.iterrows():
  print(row['updated_VS_ID'])
  coordinates = (row['updated_VS_lat'], row['updated_VS_long'])
  result = reverseGeocode(coordinates)
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'vs_loc_rg'] = result[0]['name']
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'vs_reg_rg'] = result[0]['admin1']
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'vs_county_rg'] = result[0]['admin2']
  location_data.loc[location_data['updated_VS_ID']==row['updated_VS_ID'], 'vs_country_rg'] = result[0]['cc']

100874124
Loading formatted geocoded file...
101067666
100678777
100164975
100829721
100780593
100364181
100679984
100532526
100350126
100433484
101257366
100939812
100889724
100920769
100821741
100366057
100907828
100540727
100368030
100365002
100365080
100368383
101214198
100361195
100714421
100714422
100922747
100350133
100064893
101201540
100679910
100679844
100922777
101071955
100845535
100679820
100922933
100680091
100679315
100920044
100855588
100910308
100893951
100679002
100074079
100829704
100855106
100957971
101046438
100878274
100740775
100907759
100679818
100939740
100856432
100369328
100974854
100920029
100366383
100172014
100827735
100183514
100679810
101233342
100726054
100364937
100367662
100669328
100920025
100829759
100907817
100874486
100364657
100678689
100669778
100735436
100410746
100679817
100679003
101064810
100680088
100920891
100364823
100455922
100568204
100366190
100364965
100853829
100673452
100820858
100613872
100359218
100885237
100358246
100364779
100363338
100939861
100875265
100380951
100865704
100351217
100356603
100939830
100907815
100740777
100894811
100922657
100357317
100791840
100960101
100829720
100454797
100663389
100939813
100874165
100657704
100907802
100364230
100589650
100885473
100355048
100669928
100365055
100898729
100099158
100455353
100922815
100783868
100531472
100948925
100737783
100922871
100874123
101140876
100935737
100854873
100351791
100678954
100922612
100351456
100836315
100680016
100920560
100366340
100937736
100922666
100873732

In [0]:
location_data['vs_reg_rg'] = location_data['vs_reg_rg'].str.lower()

In [0]:
location_data.loc[location_data['updated_VS_region']!=location_data['vs_reg_rg'], 'region_diff'] = 1
location_data.loc[location_data['updated_VS_region']==location_data['vs_reg_rg'], 'region_diff'] = 0

In [0]:
location_data[location_data['region_diff']==1][['updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 'vs_lat', 'vs_lon', 
                                                'updated_VS_lat', 'updated_VS_long', 'vs_loc_rg', 'vs_reg_rg']].drop_duplicates()

Out[16]:

,updated_VS_ID,updated_VS_location,updated_VS_region,vs_lat,vs_lon,updated_VS_lat,updated_VS_long,vs_loc_rg,vs_reg_rg
30,100821741,dodge,wisconsin,43.413614,-88.717783,44.134556,-91.548546,Winona,minnesota
35,100540727,washington,wisconsin,43.373499,-88.237089,45.394794,-86.931213,Escanaba,michigan
98,100878274,dakota,minnesota,43.380129,-100.527852,43.912109,-91.358299,Brice Prairie,wisconsin
149,100907817,howard,missouri,50.849241,-114.983982,38.091278,-94.583862,Pleasanton,kansas


In [0]:
location_data.loc[location_data['updated_VS_ID']==100821741, 'updated_VS_lat'] = location_data.loc[location_data['updated_VS_ID']==100821741, 'vs_lat']
location_data.loc[location_data['updated_VS_ID']==100821741, 'updated_VS_long'] = location_data.loc[location_data['updated_VS_ID']==100821741, 'vs_lon']
location_data.loc[location_data['updated_VS_ID']==100540727, 'updated_VS_lat'] = location_data.loc[location_data['updated_VS_ID']==100540727, 'vs_lat']
location_data.loc[location_data['updated_VS_ID']==100540727, 'updated_VS_long'] = location_data.loc[location_data['updated_VS_ID']==100540727, 'vs_lon']
location_data.loc[location_data['updated_VS_ID']==100878274, 'updated_VS_lat'] = 44.6666331
location_data.loc[location_data['updated_VS_ID']==100878274, 'updated_VS_long'] = -93.0668807
location_data.loc[location_data['updated_VS_ID']==100907817, 'updated_VS_lat'] = 39.1500322
location_data.loc[location_data['updated_VS_ID']==100907817, 'updated_VS_long'] = -92.6835165

In [0]:
location_data[location_data['region_diff']==1][['updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 'vs_lat', 'vs_lon', 
                                                'updated_VS_lat', 'updated_VS_long', 'vs_loc_rg', 'vs_reg_rg']].drop_duplicates()

Out[18]:

,updated_VS_ID,updated_VS_location,updated_VS_region,vs_lat,vs_lon,updated_VS_lat,updated_VS_long,vs_loc_rg,vs_reg_rg
30,100821741,dodge,wisconsin,43.413614,-88.717783,43.413614,-88.717783,Winona,minnesota
35,100540727,washington,wisconsin,43.373499,-88.237089,43.373499,-88.237089,Escanaba,michigan
98,100878274,dakota,minnesota,43.380129,-100.527852,44.666633,-93.066881,Brice Prairie,wisconsin
149,100907817,howard,missouri,50.849241,-114.983982,39.150032,-92.683516,Pleasanton,kansas


In [0]:
location_data

Out[19]:

,updated_VS_ID,updated_VS_region,updated_VS_location,vs_lat,vs_lon,EVENT_MONTH,EVENT_TYPE,Material_No.,Material_Description,updated_VS_lat,updated_VS_long,vs_loc_rg,vs_reg_rg,vs_county_rg,vs_country_rg,region_diff
0,100874124,north carolina,haywood,35.578738,-82.590945,12,cold/wind chill,43916104,tray pp round white stfr 22.6flozus,35.619850,-79.064209,Pittsboro,north carolina,Chatham County,US,0.0
1,100874124,north carolina,haywood,35.578738,-82.590945,12,cold/wind chill,22013710,tray pet rectangular stfr 13flozus,35.619850,-79.064209,Pittsboro,north carolina,Chatham County,US,0.0
2,100874124,north carolina,haywood,35.578738,-82.590945,12,cold/wind chill,43428856,clear tray pp embossed 414 1c,35.619850,-79.064209,Pittsboro,north carolina,Chatham County,US,0.0
3,100874124,north carolina,haywood,35.578738,-82.590945,12,cold/wind chill,22013712,tray pet rectangular stfr 24flozus,35.619850,-79.064209,Pittsboro,north carolina,Chatham County,US,0.0
4,101067666,indiana,dearborn,39.134034,-84.982100,12,winter weather,22002525,flour wheat soft blch unenriched,39.097061,-84.858940,Lawrenceburg,indiana,Dearborn County,US,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,100358246,pennsylvania,chester,53.159935,-2.902856,7,thunderstorm wind,43521046,chicken whtmesti 3/8x2.25ingrldckd 30x30,39.852680,-75.360687,Chester,pennsylvania,Delaware County,US,0.0
4371,100358246,pennsylvania,chester,53.159935,-2.902856,7,hail,43064838,"""chicken whtmedcd3/4"""" hltd iqf 30x30#tote""",39.852680,-75.360687,Chester,pennsylvania,Delaware County,US,0.0
4372,100358246,pennsylvania,chester,53.159935,-2.902856,7,hail,43521046,chicken whtmesti 3/8x2.25ingrldckd 30x30,39.852680,-75.360687,Chester,pennsylvania,Delaware County,US,0.0
4373,100358246,pennsylvania,chester,53.159935,-2.902856,7,flash flood,43064838,"""chicken whtmedcd3/4"""" hltd iqf 30x30#tote""",39.852680,-75.360687,Chester,pennsylvania,Delaware County,US,0.0


In [0]:
location_data_exw_final_powerbi = location_data[['updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 'updated_VS_lat', 
                                                 'updated_VS_long', 'EVENT_MONTH', 'EVENT_TYPE', 'Material_No.', 'Material_Description']].drop_duplicates()

In [0]:
location_data_exw_final_powerbi.shape

Out[21]: (4375, 9)

In [0]:
location_data_exw_final_powerbi_table=spark.createDataFrame(location_data_exw_final_powerbi)
location_data_exw_final_powerbi_table.write.saveAsTable('monthly_extreme_weather_events_data_08_02', mode = 'overwrite')

In [0]:
location_data_exw_final_powerbi['updated_VS_ID'].unique().shape

Out[23]: (148,)